# _Word Embeddings_

A **representação semântica distribuı́da** é baseada na hipótese distribucional que estabelece que o sentido de uma palavra é dado por seu contexto de ocorrência [2]. Esses vetores de palavras podem ser usados como recursos em uma variedade de aplicações, tais como: classificação de documentos [3], perguntas e respostas [4] e reconhecimento de entidade nomeada [5]. A representação de palavras como vetores contı́nuos tem uma longa história [6], [7], [8]). 

Muitos tipos diferentes de modelos foram propostos para estimar representações contı́nuas de palavras, incluindo a Análise Semântica Latente (do inglês, _Latent Semantic Analysis_ – **LSA**) e a Alocação Latente de Dirichlet (do inglês, _Latent Dirichlet Allocation_ – **LDA**). 

Já as representações distribuı́das de palavras aprendidas por redes neurais apresentam um desempenho significativamente superior ao LSA ao preservar regularidades lineares entre as palavras [1], [9]. Quanto ao LDA, sabe-se que ele é computacionalmente caro quando usado em grandes conjuntos de dados.

Neste notebook, faremos o treinamento do WordeVec em um córpus da Wikipédia no PT-BR.


In [5]:
#imports
import multiprocessing

from gensim.corpora import  WikiCorpus
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

## NNLM (_Neural Network Language Model_)

Uma arquitetura interessante de **NNLM** que foi apresentada por [10], é a que vetores de palavras são primeiro aprendidos usando uma rede neural com uma única camada oculta. Os vetores de palavras são, então, usados para treinar o NNLM. Assim, os vetores de palavra são aprendidos mesmo sem construir o NNLM completo. Mikolov et al. (2013) [1] estende diretamente essa arquitetura, focando apenas no primeiro passo onde os vetores de palavras são aprendidos usando um modelo simples. 

O objetivo é gerar vetores contendo números de tal forma que palavras similares de acordo com seus contextos estarão “próximas” no espaço vetorial, como ilustra a figura abaixo. Segundo Mikolov et al. (2013) [1], os vetores em cada lı́ngua foram projetados para 2 dimensões usando **PCA** e rotacionados manualmente para enfatizar a similaridade. Na figura temos palavras em inglês do lado direito e em espanhol do lado esquerdo.

<img src='images/word2vec_similaridade.png' width=600>

Na célula seguinte, definiremos o caminho do córpus que utilizaremos para treinar o Word2Vec e onde o modelo será salvo.

O córpus que utilizaremos está disponível em https://dumps.wikimedia.org/ptwiki/.

In [3]:
#import data
inp = "data/wiki.pt-br.text"
outp = "data/wiki.pt-br.word2vec.model"

## Word2Vec

Partindo da premissa de que técnicas básicas como contagem de n-gramas já estão em seu limite, Mikolov et al. (2013) [1] propõe a utilização de modelos de linguagem baseados em redes neurais para modelar representações distribuídas de palavras. O principal objetivo das técnicas propostas por Mikolov et al. (2013) [1] é aprender vetores de palavras de alta qualidade, a partir de enormes conjuntos de dados com bilhões de palavras. De maneira surpreendente, verificou-se que a similaridade das representações de palavras vai além das simples regularidades sintáticas. Dentro de um espaço de dimensões vetoriais, usando uma simples operação algébrica nos vetores de palavras, foi mostrado por exemplo que:

> vetor(**rei**) - vetor(**homem**) + vetor(**mulher**) = vetor que está próximo da representação vetorial da palavra **rainha**.

Mikolov et al. (2013) [1] propõe duas arquiteturas de modelos para a aprendizagem de representações distribuı́das de palavras que tentam minimizar a complexidade computacional: o modelo _Continuous Bag-of-Words_ (CBOW) e o modelo Skip-gram.

* **CBOW** – No CBOW, a arquitetura é semelhante à do NNLM _feedforward_, onde a camada escondida não-linear é removida e a camada de projeção é compartilhada para todas as palavras (não apenas a matriz de projeção). Assim, todas as palavras são projetadas na mesma posição. Essa arquitetura é chamada de modelo de saco de palavras (_bag of words_), pois a ordem das palavras não influencia a projeção. O CBOW usa representação distribuı́da contı́nua do contexto. A arquitetura do modelo é mostrada na figura abaixo, na qual pode-se observar que a matriz de pesos entre a entrada e a camada de projeção é compartilhada para todas as posições de palavras (da mesma maneira que no NNLM).
    
    
* **Skip-gram** – A arquitetura do Skip-gram é semelhante à do CBOW, mas em vez de prever a palavra atual com base no contexto, Skip-gram tenta maximizar a classificação de uma palavra com base em outra da mesma sentença. Mais precisamente, usa-se cada palavra atual como uma entrada para um classificador log-linear para prever palavras dentro de um intervalo anterior e posterior à palavra atual. O aumento do intervalo melhora a qualidade dos vetores de palavra resultantes, mas também aumenta a complexidade computacional. A distância entre uma palavra do contexto e a palavra atual indica o grau de relação entre elas. Quanto mais distante, menos relacionada estará à palavra atual, podendo receber pesos menores.

<img src='images/CBOW_Skip-Gram.png' width=500>

### Parâmetros

Na próxima célula de código, deinimos os seguintes parâmetros:

* **sg**: define o algoritmo de treinamento. Por padrão, o CBOW é usado (sg = 0). O outro é o skip-gram (sg = 1).

* **size**: dimensionalidade dos vetores.

* **window**: é a quantidade de palavras anteriores e posteriores à palavra alvo.

* **LineSentence**: Interpreta uma string ou arquivo. Cada linha é uma sentença.

* **min_count**: ignore as palavras com frequência total inferior a **min_count**.

* **max_vocab_size**: Limite a RAM durante a construção do vocabulário; se houver mais palavras únicas do que **max_vocab_size**, ocorre a poda os infrequentes. Cada 10 milhões de tipos de palavras precisam de cerca de 1GB de RAM.

* **sample**: limiar para configurar quais palavras de maior frequência são aleatoriamente reduzidas; O padrão é 1e-3, o intervalo útil é (0, 1e-5).

* **workers**: parâmetro que indica quantos cores da máquina serão utilizados para o treinamento.

* **hs**: se 1, softmax hierárquico será usado para o treinamento do modelo. Se definido como 0 (padrão), e existir amostragem negativa, esse recurso será utilizado.

* **negative**: se > 0, será utilizada amostragem negativa. O valor indica quantas "palavras de ruído" devem ser consideradas (normalmente entre 5 a 20). Se **negative** configurado para 0, não é utilizada a amostragem negativa.

* **cbow_mean**: se 0, usa a soma dos vetores das palavras de contexto. Se 1 (padrão), usa a média. Aplica-se apenas quando cbow é utilizado.

* **hashfxn**: função hash para inicializar os pesos aleatoriamente.

* **iter**: número de iterações (épocas) sobre o córpus. O padrão é 5.


In [9]:
#train model
%time model = Word2Vec(LineSentence(inp), size=400, window=5, min_count=5, workers=multiprocessing.cpu_count())

CPU times: user 1h 59min 41s, sys: 41.5 s, total: 2h 23s
Wall time: 37min 16s


* **init_sims(replace=True)**: Normaliza o modelo para não demandar tanta memória.

In [10]:
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)

Salva o modelo no caminho especificado em outp

In [11]:
model.save(outp)

## Leituras

Sugiro as seguintes leituras complementares sobre o Word2Vec.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)
* [Deep Learning com Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) por Gensim

## Referências

[1] _MIKOLOV, T.; CHEN, K.; CORRADO, G.; DEAN, J. Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781, 2013._

[2] _BRUNI, E.; TRAN, N.-K.; BARONI, M. Multimodal distributional semantics. J. Artif. Intell. Res.(JAIR), v. 49, n. 2014, p. 1–47, 2014._

[3] _SEBASTIANI, F. Machine learning in automated text categorization. ACM computing surveys (CSUR), ACM, v. 34, n. 1, p. 1–47, 2002._

[4] _TELLEX, S.; KATZ, B.; LIN, J.; FERNANDES, A.; MARTON, G. Quantitative evaluation of passage retrieval algorithms for question answering. In: ACM. Proceedings of the 26th annual international ACM SIGIR conference on Research and development in informaion retrieval. [S.l.], 2003. p. 41–47._

[5] _TURIAN, J.; RATINOV, L.; BENGIO, Y. Word representations: a simple and general method for semi-supervised learning. In: ASSOCIATION FOR COMPUTATIONAL LINGUISTICS. Proceedings of the 48th annual meeting of the association for computational linguistics. [S.l.], 2010. p. 384–394._

[6] _HINTON, G. E. Distributed representations. 1984._

[7] _HINTON, G.; RUMELHART, D.; WILLIAMS, R. Learning internal representations by back-propagating errors. Parallel Distributed Processing: Explorations in the Microstructure of Cognition, v. 1, 1985._

[8] _ELMAN, J. L. Finding structure in time. Cognitive science, Wiley Online Library, v. 14, n. 2, p. 179–211, 1990._

[9] _ZHILA, A.; YIH, W.-t.; MEEK, C.; ZWEIG, G.; MIKOLOV, T. Combining heterogeneous models for measuring relational similarity. In: HLT-NAACL. [S.l.: s.n.], 2013. p. 1000–1009._

[10] _MIKOLOV, T.; KOPECKY, J.; BURGET, L.; GLEMBEK, O. et al. Neural network based language models for highly inflective languages. In: IEEE. Acoustics, Speech and Signal Processing, 2009. ICASSP 2009. IEEE International Conference on. [S.l.], 2009. p. 4725–4728._